In [163]:
import pandas as pd
import datetime
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff


In [164]:
X=pd.read_csv('X.csv')

a1=list(X.Region.unique())
regions_dict=dict(zip(range(1,len(a1)+1),a1))
def my_date(dt):
    a= dt.split('-')
    return datetime.date(int(a[0]),int(a[1]),int(a[2]))
X.Date=X.Date.apply(my_date)
myday = list(X.tail(1).Date)[0]
print(regions_dict)

{1: 'Адыгея', 2: 'Алтай', 3: 'Алтайский край', 4: 'Амурская обл.', 5: 'Архангельская обл.', 6: 'Астраханская обл.', 7: 'Башкортостан', 8: 'Белгородская обл.', 9: 'Брянская обл.', 10: 'Бурятия', 11: 'Владимирская обл.', 12: 'Волгоградская обл.', 13: 'Вологодская обл.', 14: 'Воронежская обл.', 15: 'Дагестан', 16: 'Еврейская АО', 17: 'Забайкальский край', 18: 'Ивановская обл.', 19: 'Ингушетия', 20: 'Иркутская обл.', 21: 'Кабардино-Балкария', 22: 'Калининградская обл.', 23: 'Калмыкия', 24: 'Калужская обл.', 25: 'Камчатский край', 26: 'Карачаево-Черкессия', 27: 'Карелия', 28: 'Кемеровская обл.', 29: 'Кировская обл.', 30: 'Коми', 31: 'Костромская обл.', 32: 'Краснодарский край', 33: 'Красноярский край', 34: 'Крым', 35: 'Курганская обл.', 36: 'Курская обл.', 37: 'Ленинградская обл.', 38: 'Липецкая обл.', 39: 'Магаданская обл.', 40: 'Марий Эл', 41: 'Мордовия', 42: 'Москва', 43: 'Московская обл.', 44: 'Мурманская обл.', 45: 'Ненецкий АО', 46: 'Нижегородская обл.', 47: 'Новгородская обл.', 48: '

In [165]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13793 entries, 0 to 13792
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Date                13793 non-null  object 
 1   Confirmed           13793 non-null  int64  
 2   Recovered           13793 non-null  int64  
 3   Deaths              13793 non-null  int64  
 4   Day_deaths          13793 non-null  int64  
 5   Day_confirmed       13793 non-null  int64  
 6   Day_recovered       13793 non-null  int64  
 7   Rt                  12636 non-null  float64
 8   Rt2                 12746 non-null  float64
 9   Rate_conf           13793 non-null  float64
 10  Rate_ill            13793 non-null  float64
 11  Region              13793 non-null  object 
 12  Remaining_ill       13793 non-null  int64  
 13  MA7_dayconfirmed    13277 non-null  float64
 14  MA14_dayconfirmed   12675 non-null  float64
 15  MA7_remaining_ill   13277 non-null  float64
 16  MA14

In [166]:
covid_rates = pd.DataFrame()
mask = (X.Date==myday)&(X.Region!='Россия')
today_cases = X[mask]
#covid_rates.reset_index(drop=True)

covid_rates['По количеству текущих <br>больных'] = today_cases.sort_values(by=['Remaining_ill'],ascending = False).Region.reset_index(drop=True)
covid_rates['По количеству новых'] = today_cases.sort_values(by=['Day_confirmed'],ascending = False).Region.reset_index(drop=True)
covid_rates['По умершим за день'] = today_cases.sort_values(by=['Day_deaths'],ascending = False).Region.reset_index(drop=True)
covid_rates['По выздоровевшим<br>за день'] = today_cases.sort_values(by=['Day_recovered'],ascending = False).Region.reset_index(drop=True)
param = {'xgap':0}
fig =  ff.create_table(covid_rates.head(15),height_constant=200,**param)
fig.update_layout(
    
                  title_text ='Таблица рейтингов заболеваемости COVID-19 по регионам',
                  margin = {'t':50, 'b':100},
                  title_x = 0.5,
                  title_y= 0.95,
                  title_xanchor = "center",
                  title_yanchor = "top", 
                  
                  width = 990, height = 600,template = 'gridon',
    
    
                  xaxis_title='',yaxis_title = ''
                  )
for i in range(0,4):
    fig.layout.annotations[i].font.size = 10
fig.show()




In [167]:
region_number = 10
regions_towatch = list(covid_rates['По количеству текущих <br>больных'].head(region_number))
regions_towatch.append("Россия")
regions_towatch

['Москва',
 'Московская обл.',
 'Санкт-Петербург',
 'Челябинская обл.',
 'Свердловская обл.',
 'Саратовская обл.',
 'Иркутская обл.',
 'Волгоградская обл.',
 'Ставропольский край',
 'Ульяновская обл.',
 'Россия']

In [168]:
mask = (X.Date>datetime.date(2020,5,1))&(X.Region.isin(regions_towatch))
#mask = (X.Region.isin(regions_towatch))

fig = px.line(X[mask], y="Day_confirmed", x="Date",
                             color="Region", line_group="Region",
              line_shape="spline", render_mode="notebook",labels={'Region':'Регион:'},
              color_discrete_map={
                "Москва": "red",
                "Московская обл.": "magenta",
                "Санкт-Петербург": "goldenrod",
                "Нижегородская обл.": "darkblue",
                "Свердловская обл.": "green",
                 'Россия': 'aquamarine'},
             title="График количества новых больных по 5 регионам с наибольшим количеством заболеваний")

fig.update_layout(
    annotations=[
            dict(
            x=0.5,
            y=-0.15,
            showarrow=False,
            text="Данные: Яндекс",
            xref="paper",
            yref="paper")],
           
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                  
                  width = 990, height = 600,template = 'gridon',
    
    
                  xaxis_title='',yaxis_title = ''
)

fig.show()






In [169]:
#mask = (X.Date>datetime.date(2020,5,1))&(X.Region.isin(regions_towatch))

colors_dict = {"Москва": "red",
                "Московская обл.": "magenta",
                "Санкт-Петербург": "goldenrod",
                "Нижегородская обл.": "darkblue",
                "Свердловская обл.": "green",
                 'Россия':'aquamarine'}
fig = go.Figure()
for i in regions_towatch:
    mask = X['Region'] == i
        
    fig.add_trace(
        
        go.Bar(
        name = f'{i}',
        x=X[mask].Date,
        y=X[mask].Rt,
        text=round(X[mask].Rt,3),
        textposition='inside',
        marker_color= colors_dict.get(i),
        
        showlegend=True))




                  
fig.update_layout(
    annotations=[
        dict(
            x='2020-04-13',
            y=1.717,
            xref="x",
            yref="y",
            text="Введение<br>пропускной<br>системы в Москве",
            showarrow=True,
            arrowhead=2,
            ax=0,
            ay=-60),
        dict(
            x='2020-06-09',
            y=0.942,
            xref="x",
            yref="y",
            text="Отмена<br>пропускной<br>системы в Москве",
            showarrow=True,
            arrowhead=2,
            ax=0,
            ay=-60),
            dict(x=0.5,y=-0.15,
            showarrow=False,
            text="Данные : Яндекс <br> формула расчета из методических рекомендациях Роспотребнадзора.",
            xref="paper",
            yref="paper")],
                  title = "Коэффициент распространения (Rt) по 5 регионам ,<br> с максимальным количеством новых заболеваний ",
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                 # legend_x = 0.05,legend_y = 0.9,
                  width = 990, height = 600,template = 'gridon',
    
    
                 # xaxis_title='Коэффициент рассчитан, исходя из данных оперативного штаба и <br>' + 
                               #' формулы расчета в методических рекомендациях Роспотребнадзора. ',
    yaxis_title = ' '
)


fig.show()


In [170]:
print('Все регионы по которым есть статистика:')
print('*'*40)
print(regions_dict.items())
print('*'*40)
print('Рассматриваем регионы: ')
for i in regions_dict.keys():
    if regions_dict.get(i) in regions_towatch:
        print(i,regions_dict.get(i))


my_region = regions_dict[41]
print('Выбрали регион: ',my_region)


Все регионы по которым есть статистика:
****************************************
dict_items([(1, 'Адыгея'), (2, 'Алтай'), (3, 'Алтайский край'), (4, 'Амурская обл.'), (5, 'Архангельская обл.'), (6, 'Астраханская обл.'), (7, 'Башкортостан'), (8, 'Белгородская обл.'), (9, 'Брянская обл.'), (10, 'Бурятия'), (11, 'Владимирская обл.'), (12, 'Волгоградская обл.'), (13, 'Вологодская обл.'), (14, 'Воронежская обл.'), (15, 'Дагестан'), (16, 'Еврейская АО'), (17, 'Забайкальский край'), (18, 'Ивановская обл.'), (19, 'Ингушетия'), (20, 'Иркутская обл.'), (21, 'Кабардино-Балкария'), (22, 'Калининградская обл.'), (23, 'Калмыкия'), (24, 'Калужская обл.'), (25, 'Камчатский край'), (26, 'Карачаево-Черкессия'), (27, 'Карелия'), (28, 'Кемеровская обл.'), (29, 'Кировская обл.'), (30, 'Коми'), (31, 'Костромская обл.'), (32, 'Краснодарский край'), (33, 'Красноярский край'), (34, 'Крым'), (35, 'Курганская обл.'), (36, 'Курская обл.'), (37, 'Ленинградская обл.'), (38, 'Липецкая обл.'), (39, 'Магаданская обл.'

In [171]:
mask = X.Region ==my_region
#mask = (X.Date>datetime.date(2020,5,1)&(X.Region.isin(regions_towatch))
#mask = (X.Date>datetime.date(2020,5,1)&(X.Region==my_region)

fig = px.bar(X[mask], x='Date', 
             y="Day_confirmed",color ='Day_confirmed',
             title=f'{my_region}: '+'график количества новых заболевших COVID-19 по датам',
              color_continuous_scale= 'jet',text = 'Day_confirmed',
             labels = {'Day_confirmed' : "Новые заболевшие",'Remaining_ill':'Текущие больные'})
fig.add_trace(
    go.Scatter(
        name='14-дневная скользящая средняя',
        x=X[mask].Date,
        y=X[mask]['MA14_dayconfirmed'],
        mode="lines",
        
        line=go.scatter.Line(color="green"),
        showlegend=True)
)
fig.add_trace(
    go.Scatter(
        name = '7-дневная скользящая средняя',
        x=X[mask]['Date'],
        y=X[mask]['MA7_dayconfirmed'],
        mode="lines",
        
        line=go.scatter.Line(color="red"),
        showlegend=True)
)
fig.update_layout(
    annotations=[
#         dict(
#             x='2020-04-13',
#             y=1355,
#             xref="x",
#             yref="y",
#             text="Введение<br>пропускной<br>системы в Москве",
#             showarrow=True,
#             arrowhead=2,
#             ax=0,
#             ay=-60),
#         dict(
#             x='2020-06-09',
#             y=1572,
#             xref="x",
#             yref="y",
#             text="Отмена<br>пропускной<br>системы в Москве",
#             showarrow=True,
#             arrowhead=2,
#             ax=0,
#             ay=-90),
        dict(
            x=0.5,
            y=-0.15,
            showarrow=False,
            text="Данные: Яндекс",
            xref="paper",
            yref="paper"),
        
                ],
    
    
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                  legend_x = 0.15,legend_y = 0.95,
                  width = 990, height = 600,template = 'gridon',
    
    
                  xaxis_title='',yaxis_title = ' '
)


fig.show()




In [183]:
mask = X.Region ==my_region
#mask = (X.Date>datetime.date(2020,5,1))&(X.Region==my_region)
#Y = X[mask]

#Y['base'] = Y['Change_dayconf']*Y['Change_dayconf'].apply(lambda d:0 if d>0 else 1)
#Y['Date']= pd.to_datetime(Y.Date)
fig = go.Figure()
fig.add_trace(
              go.Bar(x = X[mask].Date,y=X[mask].Change_dayconf,text = X[mask]['Change_dayconf'],textposition = 'inside',hoverinfo='x+y',
                
                
                 marker_color = X[mask]['Change_dayconf'],marker_colorscale = 'Temps',marker_colorbar ={'tickmode':'auto',
                                    'title':{'text':'Изменение','side':'top'}}
                    )   
                )
fig.update_layout(
   annotations=[
        
        dict(x=0.5,y=-0.15,
            showarrow=False,
            text="",
            xref="paper",
            yref="paper")],
                  title=f'{my_region}: динамика новых заболевших COVID-19 по дням<br>(изменение относительно предыдущего дня)',
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                  legend_x = 0.05,legend_y = 0.98,
                  width = 990, height = 600,template = 'gridon',
    
    
                  xaxis_title='Данные: Яндекс',
    yaxis_title = ' '
)

fig.show()






In [173]:
mask = X.Region ==my_region
#mask = (X.Date>'2020-05-01')&(X.Region.isin(regions_towatch))

fig = px.bar(X[mask], x='Date', y="Remaining_ill",color ='Remaining_ill',
             title=f'{my_region}: '+'график количества текущих больных COVID-19 по датам',
              color_continuous_scale= 'tempo',text = 'Remaining_ill',
            labels = {'Day_confirmed' : "Новые заболевшие",'Remaining_ill':'Текущие больные'})
fig.add_trace(
    go.Scatter(
        name='14-дневная скользящая средняя',
        x=X[mask].Date,
        y=X[mask]['MA14_remaining_ill'],
        mode="lines",
        
        line=go.scatter.Line(color="green"),
        showlegend=True)
)

fig.add_trace(
    go.Scatter(
        name = '7-дневная скользящая средняя',
        x=X[mask].Date,
        y=X[mask]['MA7_remaining_ill'],
        mode="lines",
        
        line=go.scatter.Line(color="red"),
        showlegend=True)
)
fig.update_layout(
    annotations=[
#         dict(
#             x='2020-04-13',
#             y=10594,
#             xref="x",
#             yref="y",
#             text="Введение<br>пропускной<br>системы в Москве",
#             showarrow=True,
#             arrowhead=2,
#             ax=0,
#             ay=-60),
#         dict(
#             x='2020-06-09',
#             y=85824,
#             xref="x",
#             yref="y",
#             text="Отмена<br>пропускной<br>системы в Москве",
#             showarrow=True,
#             arrowhead=2,
#             ax=0,
#             ay=-60),
        
        dict(x=0.5,y=-0.15,
            showarrow=False,
            text="",
            xref="paper",
            yref="paper")],
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                  legend_x = 0.05,legend_y = 0.98,
                  width = 990, height = 600,template = 'gridon',
    
    
                  xaxis_title='Текущие больные = количество выявленных - количество выздоровевших - количество умерших'+
                                '<br>Данные: Яндекс',
    yaxis_title = ' '
)


fig.show()





In [184]:
mask = X.Region ==my_region
#mask = (X.Date>'2020-05-01')&(X.Region==my_region)
Y = X[mask]

#Y['base'] = Y['Change_dayconf']*Y['Change_dayconf'].apply(lambda d:0 if d>0 else 1)
Y['Date']= pd.to_datetime(Y.Date)
fig = go.Figure()
fig.add_trace(
              go.Bar(x = Y.Date,y=Y.Change_remill,text = Y['Change_remill'],textposition = 'inside',hoverinfo='x+y',
                
                #marker_color=Y['Change_remill'].apply(lambda d:'crimson' if d>0 else 'green'),
                marker_color = Y['Change_remill'],marker_colorscale = 'Temps',marker_colorbar ={'tickmode':'auto',
                                    'title':{'text':'Изменение','side':'top'}})     
                )
fig.update_layout(
    annotations=[
#         dict(
#             x='2020-04-13',
#             y=10594,
#             xref="x",
#             yref="y",
#             text="Введение<br>пропускной<br>системы в Москве",
#             showarrow=True,
#             arrowhead=2,
#             ax=0,
#             ay=-60),
#         dict(
#             x='2020-06-09',
#             y=-3877,
#             xref="x",
#             yref="y",
#             text="Отмена<br>пропускной<br>системы в Москве",
#             showarrow=True,
#             arrowhead=2,
#             ax=-20,
#             ay=40),
#         dict(
#             x='2020-05-20',
#             y=-2816,
#             xref="x",
#             yref="y",
#             text="20 мая",
#             showarrow=True,
#             arrowhead=2,
#             ax=-20,
#             ay=40),
        
        dict(x=0.5,y=-0.15,
            showarrow=False,
            text="",
            xref="paper",
            yref="paper")],
                  title=f'{my_region}: динамика текущих больных COVID-19 по дням<br>(изменение относительно предыдущего дня)',
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                  legend_x = 0.05,legend_y = 0.98,
                  width = 990, height = 600,template = 'gridon',
    
    
                  xaxis_title='Текущие больные = количество выявленных - количество выздоровевших - количество умерших'+
                               '<br>Данные : Яндекс',
    yaxis_title = ' '
)

fig.show()


In [175]:
delta = datetime.timedelta(14)
xday=myday-delta
print(xday)

2020-08-04


In [176]:
delta = datetime.timedelta(14)
past14=myday-delta
mask = (X.Date>=past14)&(X.Region.isin(regions_towatch))

def my_round(k):
    return round(k,3)

# #mask = (X.Date>='2020-06-04')

Z = X[mask].groupby(['Region','Date']).Rt.sum().apply(my_round).unstack().reset_index()
Z.dropna(inplace=True)

d = Z.drop('Region',axis = 1).values

# my_map= {'Москва':'2.Москва','Московская обл.':'3.Московская обл.','Нижегородская обл.':'5.Нижегородская обл.',
#                     'Россия':'1.Россия', 'Санкт-Петербург':'4.Санкт-Петербург'}
# Z.Region = Z.Region.map(my_map)

print(d)
Z.tail(10)
print(regions_towatch)

[[1.03  1.025 1.035 1.016 1.    1.024 1.003 1.018 1.037 1.    1.003 1.
  0.995 0.987 0.966]
 [0.929 0.936 0.936 0.931 0.905 0.901 0.911 0.917 0.949 0.956 0.951 0.94
  0.938 0.952 0.968]
 [1.007 1.    1.01  1.002 1.004 1.005 1.002 1.007 1.007 1.007 1.001 0.999
  0.999 0.997 1.001]
 [1.126 1.103 1.07  1.039 1.021 0.98  0.967 0.964 0.95  0.968 0.982 0.987
  1.005 1.015 0.992]
 [0.981 0.966 0.961 0.959 0.976 0.987 0.987 0.98  0.973 0.967 0.972 0.991
  0.99  0.988 0.975]
 [0.982 0.985 0.981 0.98  0.977 0.991 0.994 0.998 1.013 1.006 1.013 1.021
  1.014 1.013 1.006]
 [0.967 0.962 0.956 0.969 0.987 0.997 1.024 1.024 1.021 1.013 0.997 1.005
  1.008 1.013 1.026]
 [0.909 0.892 0.898 0.9   0.927 0.956 0.972 0.977 0.967 0.932 0.903 0.904
  0.894 0.892 0.919]
 [0.96  0.95  0.991 0.973 1.099 1.105 1.105 1.139 0.958 0.901 0.895 0.914
  1.072 1.188 1.17 ]
 [1.017 0.993 1.014 0.991 0.949 0.995 0.946 1.005 1.005 0.943 0.953 0.931
  0.951 1.01  1.057]
 [0.889 0.9   0.895 0.878 0.861 0.854 0.867 0.897 0.94

In [177]:
x1=list(Z.columns[1:])
x2=[]
for i in x1:
    x2.append(str(i.day)+'.'+str(i.month)+"."+str(i.year))
    
y=list(Z.Region)
print(list(x1))
print('-'*40)
print(x2)

[datetime.date(2020, 8, 4), datetime.date(2020, 8, 5), datetime.date(2020, 8, 6), datetime.date(2020, 8, 7), datetime.date(2020, 8, 8), datetime.date(2020, 8, 9), datetime.date(2020, 8, 10), datetime.date(2020, 8, 11), datetime.date(2020, 8, 12), datetime.date(2020, 8, 13), datetime.date(2020, 8, 14), datetime.date(2020, 8, 15), datetime.date(2020, 8, 16), datetime.date(2020, 8, 17), datetime.date(2020, 8, 18)]
----------------------------------------
['4.8.2020', '5.8.2020', '6.8.2020', '7.8.2020', '8.8.2020', '9.8.2020', '10.8.2020', '11.8.2020', '12.8.2020', '13.8.2020', '14.8.2020', '15.8.2020', '16.8.2020', '17.8.2020', '18.8.2020']


In [181]:
fig = ff.create_annotated_heatmap(d, x=x2, y=y, annotation_text=d,hoverinfo='z',
                                  colorscale='Temps')

fig.update_layout(
                  
                  
                  width = 990, height = 600,    
    
                  xaxis_title='Динамика коэффициента распространения Rt по регионам',
               #   yaxis_title = 'Регион '
)

fig.show()



In [130]:
mask = (X.Date>=past14)&(X.Region.isin(regions_towatch))


# #mask = (X.Date>='2020-06-04')

Z = X[mask].groupby(['Region','Date']).Rt2.sum().apply(my_round).unstack().reset_index()
Z.dropna(inplace=True)
d = Z.drop('Region',axis = 1).values

# my_map= {'Москва':'2.Москва','Московская обл.':'3.Московская обл.','Нижегородская обл.':'5.Нижегородская обл.',
#                     'Россия':'1.Россия', 'Санкт-Петербург':'4.Санкт-Петербург'}
# Z.Region = Z.Region.map(my_map)

print(d)
Z.head(10)

[[1.041 1.041 1.041 1.041 1.04  1.04  1.04  1.04  1.04  1.039 1.039 1.038
  1.038 1.038 1.037]
 [1.044 1.042 1.041 1.04  1.039 1.037 1.036 1.035 1.034 1.034 1.033 1.032
  1.031 1.031 1.03 ]
 [1.011 1.011 1.011 1.011 1.011 1.011 1.011 1.011 1.011 1.011 1.011 1.011
  1.011 1.011 1.011]
 [1.01  1.01  1.01  1.01  1.01  1.01  1.01  1.01  1.01  1.009 1.009 1.009
  1.009 1.009 1.009]
 [1.026 1.026 1.025 1.025 1.025 1.024 1.024 1.024 1.024 1.023 1.023 1.023
  1.023 1.022 1.022]
 [1.021 1.02  1.02  1.02  1.02  1.02  1.02  1.019 1.019 1.019 1.019 1.019
  1.019 1.019 1.019]
 [1.042 1.041 1.04  1.039 1.039 1.038 1.038 1.038 1.038 1.038 1.037 1.037
  1.037 1.036 1.036]
 [1.04  1.039 1.037 1.036 1.035 1.034 1.034 1.033 1.033 1.032 1.031 1.03
  1.029 1.028 1.027]
 [1.056 1.054 1.053 1.052 1.053 1.054 1.054 1.055 1.054 1.052 1.05  1.048
  1.048 1.05  1.051]
 [1.048 1.047 1.047 1.046 1.045 1.044 1.043 1.043 1.042 1.041 1.04  1.039
  1.038 1.038 1.038]
 [1.041 1.039 1.038 1.036 1.035 1.033 1.032 1.03  1

Date,Region,2020-08-04,2020-08-05,2020-08-06,2020-08-07,2020-08-08,2020-08-09,2020-08-10,2020-08-11,2020-08-12,2020-08-13,2020-08-14,2020-08-15,2020-08-16,2020-08-17,2020-08-18
0,Волгоградская обл.,1.041,1.041,1.041,1.041,1.040,1.040,1.040,1.040,1.040,1.039,1.039,1.038,1.038,1.038,1.037
1,Иркутская обл.,1.044,1.042,1.041,1.040,1.039,1.037,1.036,1.035,1.034,1.034,1.033,1.032,1.031,1.031,1.030
2,Москва,1.011,1.011,1.011,1.011,1.011,1.011,1.011,1.011,1.011,1.011,1.011,1.011,1.011,1.011,1.011
3,Московская обл.,1.010,1.010,1.010,1.010,1.010,1.010,1.010,1.010,1.010,1.009,1.009,1.009,1.009,1.009,1.009
4,Россия,1.026,1.026,1.025,1.025,1.025,1.024,1.024,1.024,1.024,1.023,1.023,1.023,1.023,1.022,1.022
5,Санкт-Петербург,1.021,1.020,1.020,1.020,1.020,1.020,1.020,1.019,1.019,1.019,1.019,1.019,1.019,1.019,1.019
6,Саратовская обл.,1.042,1.041,1.040,1.039,1.039,1.038,1.038,1.038,1.038,1.038,1.037,1.037,1.037,1.036,1.036
7,Свердловская обл.,1.040,1.039,1.037,1.036,1.035,1.034,1.034,1.033,1.033,1.032,1.031,1.030,1.029,1.028,1.027
8,Ставропольский край,1.056,1.054,1.053,1.052,1.053,1.054,1.054,1.055,1.054,1.052,1.050,1.048,1.048,1.050,1.051
9,Ульяновская обл.,1.048,1.047,1.047,1.046,1.045,1.044,1.043,1.043,1.042,1.041,1.040,1.039,1.038,1.038,1.038


In [131]:
x1=list(Z.columns[1:])
x2=[]
for i in x1:
    x2.append(str(i.day)+'.'+str(i.month)+"."+str(i.year))
    
y=list(Z.Region)
print(list(x1))
print(x2)

[datetime.date(2020, 8, 4), datetime.date(2020, 8, 5), datetime.date(2020, 8, 6), datetime.date(2020, 8, 7), datetime.date(2020, 8, 8), datetime.date(2020, 8, 9), datetime.date(2020, 8, 10), datetime.date(2020, 8, 11), datetime.date(2020, 8, 12), datetime.date(2020, 8, 13), datetime.date(2020, 8, 14), datetime.date(2020, 8, 15), datetime.date(2020, 8, 16), datetime.date(2020, 8, 17), datetime.date(2020, 8, 18)]
['4.8.2020', '5.8.2020', '6.8.2020', '7.8.2020', '8.8.2020', '9.8.2020', '10.8.2020', '11.8.2020', '12.8.2020', '13.8.2020', '14.8.2020', '15.8.2020', '16.8.2020', '17.8.2020', '18.8.2020']


In [132]:
fig = ff.create_annotated_heatmap(d, x=x2, y=y, annotation_text=d,
                                  colorscale='Temps')

fig.update_layout(
                  
                  
                  width = 990, height = 600,    
    
                  xaxis_title='Динамика коэффициента распространения Rt2 по регионам',
               #   yaxis_title = 'Регион '
)

fig.show()


In [133]:
mask = (X.Date>=past14)&(X.Region.isin(regions_towatch))

Z = X[mask].groupby(['Region','Date'])['Day_confirmed'].sum().unstack().reset_index()
Z.dropna(inplace=True)
d = Z.drop('Region',axis = 1).values
Z.head()

Date,Region,2020-08-04,2020-08-05,2020-08-06,2020-08-07,2020-08-08,2020-08-09,2020-08-10,2020-08-11,2020-08-12,2020-08-13,2020-08-14,2020-08-15,2020-08-16,2020-08-17,2020-08-18
0,Волгоградская обл.,92,96,98,93,88,106,97,95,91,102,97,96,92,95,89
1,Иркутская обл.,137,135,128,130,119,123,120,124,119,115,110,113,118,114,108
2,Москва,691,687,684,686,691,689,694,694,689,692,688,695,688,690,693
3,Московская обл.,161,162,159,158,161,149,153,154,152,148,156,153,154,153,145
4,Россия,5159,5204,5267,5241,5212,5189,5118,4945,5102,5057,5065,5061,4969,4892,4748


In [134]:



x1=list(Z.columns[1:])
y=list(Z.Region)
x2=[]
for i in x1:
    x2.append(str(i.day)+'.'+str(i.month)+"."+str(i.year))
    
print(x2)


['4.8.2020', '5.8.2020', '6.8.2020', '7.8.2020', '8.8.2020', '9.8.2020', '10.8.2020', '11.8.2020', '12.8.2020', '13.8.2020', '14.8.2020', '15.8.2020', '16.8.2020', '17.8.2020', '18.8.2020']


In [135]:
fig = ff.create_annotated_heatmap(d, x=x2, y=y, annotation_text=d,
                                  colorscale='Temps')

fig.update_layout(
                  
                  
                  width = 990, height = 600,
    
    
                  xaxis_title='Динамика новых заболевших по регионам',
               #   yaxis_title = 'Регион '
)

fig.show()

In [136]:
mask = (X.Date==myday)&(X.Region!='Россия')
#mask = (X.Date==myday)
#Делаем выборку по доле регина в общих случаях выявления заболеваний по России
df=X[mask].sort_values(by=['Rate_conf'],ascending = False)
#Далее заменяем все имена если регион набрал менее 1% в общих случаях по России
df2 = X[mask].reset_index(drop=True)
print(len(df2))

for i in range(len(df2)):
    if df2.loc[i,'Rate_conf']<1:
        df2.loc[i,'Region']='Регионы менее 1%'
df.head(15)

85


,Date,Confirmed,Recovered,Deaths,Day_deaths,Day_confirmed,Day_recovered,Rt,Rt2,Rate_conf,Rate_ill,Region,Remaining_ill,MA7_dayconfirmed,MA14_dayconfirmed,MA7_remaining_ill,MA14_remaining_ill,Change_dayconf,Change_remill
6731,2020-08-18,253757,199117,4677,11,693,1155,1.001,1.011,27.212751,28.715523,Москва,49963,690.714286,690.000000,51139.142857,52468.571429,3,-473
6901,2020-08-18,66233,49785,1151,5,145,118,0.992,1.009,7.102788,8.791733,Московская обл.,15297,151.571429,154.071429,15273.285714,15340.785714,-8,22
9637,2020-08-18,34325,24707,2276,19,157,83,1.006,1.019,3.680993,4.219710,Санкт-Петербург,7342,160.142857,158.714286,7121.000000,6869.714286,-2,55
7382,2020-08-18,25994,22819,436,3,116,115,0.966,1.020,2.787581,1.574201,Нижегородская обл.,2739,123.714286,117.785714,2733.571429,2895.214286,-4,-2
10277,2020-08-18,23695,19176,376,2,149,74,0.919,1.027,2.541038,2.381130,Свердловская обл.,4143,154.428571,166.714286,3927.857143,3648.571429,8,73
12517,2020-08-18,18664,16663,151,1,72,146,0.772,1.021,2.001516,1.063261,ХМАО – Югра,1850,95.714286,117.428571,2166.428571,2355.214286,-2,-75
3196,2020-08-18,15664,11477,237,9,108,226,0.968,1.030,1.679798,2.270206,Иркутская обл.,3950,113.857143,119.714286,3968.142857,4064.785714,-6,-127
5280,2020-08-18,15402,12614,392,5,117,71,0.967,1.032,1.651701,1.377067,Красноярский край,2396,119.142857,123.000000,2208.428571,1980.571429,-1,41
9152,2020-08-18,15251,12516,293,5,128,132,1.016,1.035,1.635508,1.403505,Ростовская обл.,2442,127.142857,127.642857,2367.142857,2408.357143,1,-9
2236,2020-08-18,12847,11307,97,0,95,205,1.019,1.031,1.377705,0.829344,Воронежская обл.,1443,94.571429,95.285714,1581.000000,1715.214286,-2,-110


In [137]:



fig = px.pie(df2, values='Confirmed', names='Region',color_discrete_sequence=px.colors.sequential.Rainbow,
            title=f'Всего выявленных случаев заболевания COVID-19 в России: {df2.Confirmed.sum()}<br>'+
            f'(на дату: {myday})'
            ,
            width = 900, height=1000)

fig.update_traces(textinfo='percent+label')
fig.update_layout(
        
                #  title=f'{my_region}: динамика текущих больных COVID-19 по дням<br>(изменение относительно предыдущего дня)',
                  title_x = 0.5,
                  margin = {'t':150, 'b':0},
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                 # legend_x = 0.05,legend_y = 0.98,
                     
)

fig.show()


In [138]:
mask = (X.Date==myday)&(X.Region!='Россия')
#mask = (X.Date==myday)
#Делаем выборку по доле регина в общих случаях выявления заболеваний по России
df=X[mask].sort_values(by=['Rate_ill'],ascending = False)
#Далее заменяем все имена если регион набрал менее 1% в общих случаях по России
df2 = X[mask].reset_index(drop=True)
print(len(df2))

for i in range(len(df2)):
    if df2.loc[i,'Rate_ill']<1:
        df2.loc[i,'Region']='Регионы менее 1%'
df.head(20)

85


,Date,Confirmed,Recovered,Deaths,Day_deaths,Day_confirmed,Day_recovered,Rt,Rt2,Rate_conf,Rate_ill,Region,Remaining_ill,MA7_dayconfirmed,MA14_dayconfirmed,MA7_remaining_ill,MA14_remaining_ill,Change_dayconf,Change_remill
6731,2020-08-18,253757,199117,4677,11,693,1155,1.001,1.011,27.212751,28.715523,Москва,49963,690.714286,690.000000,51139.142857,52468.571429,3,-473
6901,2020-08-18,66233,49785,1151,5,145,118,0.992,1.009,7.102788,8.791733,Московская обл.,15297,151.571429,154.071429,15273.285714,15340.785714,-8,22
9637,2020-08-18,34325,24707,2276,19,157,83,1.006,1.019,3.680993,4.219710,Санкт-Петербург,7342,160.142857,158.714286,7121.000000,6869.714286,-2,55
12997,2020-08-18,12804,8368,101,2,67,127,0.864,1.025,1.373093,2.491480,Челябинская обл.,4335,77.571429,84.000000,4374.714286,4381.214286,-2,-62
10277,2020-08-18,23695,19176,376,2,149,74,0.919,1.027,2.541038,2.381130,Свердловская обл.,4143,154.428571,166.714286,3927.857143,3648.571429,8,73
9797,2020-08-18,11116,6949,67,1,98,158,1.026,1.036,1.192073,2.356417,Саратовская обл.,4100,96.285714,95.142857,4031.571429,3944.571429,2,-61
3196,2020-08-18,15664,11477,237,9,108,226,0.968,1.030,1.679798,2.270206,Иркутская обл.,3950,113.857143,119.714286,3968.142857,4064.785714,-6,-127
1916,2020-08-18,10852,6997,80,1,89,135,0.966,1.037,1.163762,2.169628,Волгоградская обл.,3775,94.571429,95.357143,3704.714286,3653.428571,-6,-47
10917,2020-08-18,10345,6903,200,3,128,64,1.170,1.051,1.109392,1.863293,Ставропольский край,3242,120.857143,120.500000,3011.000000,2867.785714,-4,61
12357,2020-08-18,10934,7807,73,0,95,97,1.057,1.038,1.172556,1.755243,Ульяновская обл.,3054,97.000000,100.571429,2949.857143,2878.142857,-7,-2


In [139]:


fig = px.pie(df2, values='Remaining_ill', names='Region',color_discrete_sequence=px.colors.sequential.Rainbow,
            title=f'Всего текущих больных COVID-19 в России: {df2.Remaining_ill.sum()}<br>'+
            f'(на дату: {myday})'
             ,
            width = 900, height=1000)

fig.update_traces(textinfo='percent+label')
fig.update_layout(
        
                #  title=f'{my_region}: динамика текущих больных COVID-19 по дням<br>(изменение относительно предыдущего дня)',
                  margin = {'t':150, 'b':0},
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                 # legend_x = 0.05,legend_y = 0.98,
                     
)

fig.show()


In [140]:
fig_r8 = go.Figure(data=[go.Pie(labels=df2.Region, values=df2.Remaining_ill, textinfo='label+percent',
                             insidetextorientation='radial'
                            )])
fig_r8.update_layout(
                title=f'Всего текущих больных COVID-19 в России: {df2.Remaining_ill.sum()}<br>'+
                f'(на дату: {myday})',
                 margin = {'t':120, 'b':0},
                  title_x = 0.5,
                  title_y= 0.9,
                  width = 900, height=900,
                  title_xanchor = "center",
                  title_yanchor = "bottom")
fig_r8.show()

In [141]:
delta2 = datetime.timedelta(7)
past7=myday-delta2
mask = (X.Date>=past7)&(X.Region.isin(regions_towatch))&(X.Region!='Россия')
                                      
box_cases1 = X[mask].groupby(['Region','Date'])['Day_confirmed'].sum()
box_cases2 = X[mask].groupby(['Region','Date'])['Day_recovered'].sum()
for i in regions_dict.keys():
    if regions_dict.get(i) in regions_towatch:
        print(i,regions_dict.get(i))

12 Волгоградская обл.
20 Иркутская обл.
42 Москва
43 Московская обл.
56 Россия
60 Санкт-Петербург
61 Саратовская обл.
64 Свердловская обл.
68 Ставропольский край
77 Ульяновская обл.
81 Челябинская обл.


In [142]:
new_df1=box_cases1.unstack().T
new_df1.tail(20)

Region,Волгоградская обл.,Иркутская обл.,Москва,Московская обл.,Санкт-Петербург,Саратовская обл.,Свердловская обл.,Ставропольский край,Ульяновская обл.,Челябинская обл.
Date,,,,,,,,,,
2020-08-11,95,124,694,154,157,94,168,111,115,85
2020-08-12,91,119,689,152,161,96,167,95,91,88
2020-08-13,102,115,692,148,160,95,162,105,94,84
2020-08-14,97,110,688,156,159,94,154,130,86,80
2020-08-15,96,113,695,153,162,97,158,127,120,76
2020-08-16,92,118,688,154,163,98,150,129,91,79
2020-08-17,95,114,690,153,159,96,141,132,102,69
2020-08-18,89,108,693,145,157,98,149,128,95,67


In [185]:

cols = list(new_df1.columns)
color_dict=dict(zip(cols,['forestgreen','darkblue','goldenrod','magenta','hotpink','grey','maroon','coral','darkorange','lightpink']))
#cols = cols[:-1]

fig = go.Figure()
for i in cols:
    fig.add_trace(
    go.Box(x=new_df1[i],name = i,boxmean = True,marker_color = color_dict.get(i)))

fig.update_layout(title_text="Статистика по новым заболевшим за последнюю неделю",
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "top",
                  xaxis_title='Новые заболевшие', yaxis_title='Регионы',width = 990, height = 600)
fig.show()

In [186]:
new_df2=box_cases2.unstack().T
new_df2.tail(20)

Region,Волгоградская обл.,Иркутская обл.,Москва,Московская обл.,Санкт-Петербург,Саратовская обл.,Свердловская обл.,Ставропольский край,Ульяновская обл.,Челябинская обл.
Date,,,,,,,,,,
2020-08-11,135,194,1043,160,78,62,82,57,95,8
2020-08-12,87,210,1088,215,93,120,83,162,114,128
2020-08-13,91,182,787,150,96,93,78,79,73,102
2020-08-14,68,146,1350,159,72,81,75,59,126,105
2020-08-15,97,109,1275,156,76,60,77,69,89,98
2020-08-16,16,16,1045,138,53,19,76,22,23,104
2020-08-17,12,37,819,148,69,17,71,28,15,3
2020-08-18,135,226,1155,118,83,158,74,64,97,127


In [187]:

cols = list(new_df2.columns)
#cols = cols[:-1]

fig = go.Figure()
for i in cols:
    fig.add_trace(
    go.Box(x=new_df2[i],name = i,boxmean=True,marker_color = color_dict.get(i)))

fig.update_layout(title_text="Статистика по выздоровевшим за последнюю неделю",
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "top",
                  xaxis_title='Выздоровевшие', yaxis_title='Регионы',width = 990, height = 600)
fig.show()

In [188]:
from plotly.subplots import make_subplots

# fig = tools.make_subplots(rows = 1, cols = 2, specs = [[{}, {}]],
#                           horizontal_spacing = 0.05)


fig = make_subplots(rows=2, cols=1,specs = [[{}],[{}]],vertical_spacing = 0.04,shared_xaxes=True,
                  subplot_titles=("Box-график распределения по новым заболевшим(в день) за неделю:",
                                 "Box-график распределения по выздоровевшим(в день) за неделю:"))
#subplot_titles=("Статистика по новым заболевшим за последние 2 недели","Статистика по выздоровевшим за последние 2 недели")

for i in cols:
    fig.add_trace(
    go.Box(x=new_df1[i],name = i,boxmean = True,marker_color = color_dict.get(i)),row=1,col=1)

for i in cols:
    fig.add_trace(
    go.Box(x=new_df2[i],name = i,boxmean=True,marker_color = color_dict.get(i)),row=2,col=1)


fig.update_layout(
#     annotations=[
#         dict(
#             x=0.5,
#             y=-0.15,
#             showarrow=False,
#             text="Ветикальная пунктирная линия - среднее, Вертикальная сплошная - медиана распределения",
#             xref="paper",
#             yref="paper"),
        
#                 ],
    
    
                  width = 990, height = 1200,
    
    
                 showlegend=False,template ='ggplot2'
)

#fig.update_layout(height=1000, width=800,showlegend=False)
fig.show()